In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

In [15]:
pd.set_option('display.max_rows', 1000)

In [3]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

In [4]:
week_nums = [190608, 190615, 190622, 190629]
df = get_data(week_nums)

In [13]:
## reset indexes only once! 
## To avoid reseting the index again, comment out
df.reset_index(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823385 entries, 0 to 823384
Data columns (total 15 columns):
 #   Column                                                                Non-Null Count   Dtype         
---  ------                                                                --------------   -----         
 0   index                                                                 823385 non-null  int64         
 1   C/A                                                                   823385 non-null  object        
 2   UNIT                                                                  823385 non-null  object        
 3   SCP                                                                   823385 non-null  object        
 4   STATION                                                               823385 non-null  object        
 5   LINENAME                                                              823385 non-null  object        
 6   DIVISION                    

In [6]:
## Check for duplicates, and none of them are duplicates
## If there is, drop them by using: df = df.drop_duplicates()
df.duplicated().value_counts() 

False    823385
dtype: int64

In [9]:
df['DATETIME'] = pd.to_datetime(df['DATE'] + " " + df['TIME'])

In [10]:
## Need to divide up the 4 hour or any-hour window (inconsistent throughout the data) into hour for heatmap later
def time_to_hour(time):
    return int(str(time)[:2])

df['HOUR'] = df['TIME'].map(time_to_hour)

In [8]:
## Get the day of the week (for avoiding tourist traffic later)
def weekday_num(date):
    ## 2019/06/01 = 5 (Sat)
    ## 1 = Tuesday
    ## 0 = Monday
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    num = datetime.datetime.strptime(date, '%m/%d/%Y').weekday()
    return weekdays[num]

df['DAY_OF_WEEK'] = df["DATE"].apply(lambda x: weekday_num(x))

## add weekday vs weekend

## Dropping previous column since it's not needed now
## df.drop(columns=["WEEKDAY_NUM"], inplace = True)

In [16]:
df.head(1000)

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,DAY_OF_WEEK,HOUR
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,00:00:00,REGULAR,7080105,2398899,2019-06-01 00:00:00,Saturday,0
1,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,04:00:00,REGULAR,7080137,2398904,2019-06-01 04:00:00,Saturday,4
2,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,08:00:00,REGULAR,7080161,2398937,2019-06-01 08:00:00,Saturday,8
3,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,12:00:00,REGULAR,7080276,2399025,2019-06-01 12:00:00,Saturday,12
4,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,16:00:00,REGULAR,7080520,2399084,2019-06-01 16:00:00,Saturday,16
5,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/01/2019,20:00:00,REGULAR,7080817,2399121,2019-06-01 20:00:00,Saturday,20
6,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/02/2019,00:00:00,REGULAR,7081001,2399154,2019-06-02 00:00:00,Sunday,0
7,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/02/2019,04:00:00,REGULAR,7081021,2399160,2019-06-02 04:00:00,Sunday,4
8,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/02/2019,08:00:00,REGULAR,7081035,2399179,2019-06-02 08:00:00,Sunday,8
9,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/02/2019,12:00:00,REGULAR,7081119,2399230,2019-06-02 12:00:00,Sunday,12


In [ ]:
## Create unique identifier for station and line 
## because some stations have the same name serving different line
df["STATION_ID"] = df["STATION"] + "_" + df["LINENAME"]

In [ ]:
df.head(100)

In [18]:
# convert entries_diff abs
df['ENTRIES_DIFF_ABS'] = abs(df['ENTRIES_DIFF'])

KeyError: 'ENTRIES_DIFF'